In [1]:
import numpy as np
import pandas as pd;
import matplotlib.pyplot as plt

In [2]:
# python GenerateTrainingExamples.py -t 500 --output ../Activity_recognition_exp/train/Phone-Acc-nexus4_1-b_train.csv ../Activity_recognition_exp/split/Phone-Acc-nexus4_1-b.csv
sourceData = "../Activity_recognition_exp/train/Phone-Acc-nexus4_1-a_train.csv"
targetData = "../Activity_recognition_exp/train/Phone-Acc-nexus4_1-b_train.csv"

In [3]:
Ds = pd.read_csv(sourceData)
Dt = pd.read_csv(targetData)

/home/karan/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def genData(data):
    previousClassLabel = str(data.get_value(data.index[0], 'gt'))
    pos = 0
    window = 500
    t = []
    xt = []
    yt = []
    zt = []
    l = []
    while pos < data.shape[0]:
        # Make l label.
        if str(data.iloc[pos]['gt']) == 'walk':
            l.append(1)
        else:
            l.append(-1)

        # Make X row.
        xt.append(data.iloc[pos:pos + window]['x'])
        yt.append(data.iloc[pos:pos + window]['y'])
        zt.append(data.iloc[pos:pos + window]['z'])
        t.append(data.iloc[pos:pos + window]['Arrival_Time'])

        # Move to the next window
        pos += window
    return np.array(t), np.array(xt), np.array(yt), np.array(zt), np.array(l)

In [50]:
ts, xts, yts, zts, ls = genData(Ds)

/home/karan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [51]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic

In [54]:
def genGaussianFeatureSpace(kernel, t, x, y, z, left = 245, right = 255):
    finalFeatures = []
    for i in range(left,right):
        # kernel = RationalQuadratic()
        print "Done: ", i
        finalFeatures.append([])
        gpx = GaussianProcessRegressor(kernel,optimizer='fmin_l_bfgs_b',\
                                       copy_X_train=False, normalize_y=True)
        gpx.fit(t[i,:].reshape(-1, 1) - t[i,0], x[i,:])
        
        gpy = GaussianProcessRegressor(kernel,optimizer='fmin_l_bfgs_b',\
                                       copy_X_train=False, normalize_y=True)
        gpy.fit(t[i,:].reshape(-1, 1) - t[i,0], y[i,:])
        
        gpz = GaussianProcessRegressor(kernel,optimizer='fmin_l_bfgs_b',\
                                       copy_X_train=False, normalize_y=True)
        gpz.fit(t[i,:].reshape(-1, 1) - t[i,0], z[i,:])
        
        px = gpx.kernel_
        py = gpy.kernel_
        pz = gpz.kernel_
        [finalFeatures[-1].append(x_) for x_ in px.theta]
        [finalFeatures[-1].append(x_) for x_ in py.theta]
        [finalFeatures[-1].append(x_) for x_ in pz.theta]
    return np.array(finalFeatures)

In [55]:
kernel = RationalQuadratic()
left = 245
right = 255
Xmod = genGaussianFeatureSpace(kernel, ts, xts, yts, zts, left, right)
ymod = ls[left:right]

Done:  245
Done:  246
Done:  247


/home/karan/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 7690.75381333,  -717.35731665]), 'nit': 2, 'funcalls': 49}
  " state: %s" % convergence_dict)
/home/karan/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-4791.31873824,   458.82355729]), 'nit': 2, 'funcalls': 51}
  " state: %s" % convergence_dict)
/home/karan/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 797.97228688,   27.39624588]), 'nit': 3, 'funcalls': 56}
  " state: %s" % convergence_dict)


Done:  248


/home/karan/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 148.00817118,   16.39244372]), 'nit': 0, 'funcalls': 21}
  " state: %s" % convergence_dict)
/home/karan/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 814.42873389, -635.63226167]), 'nit': 2, 'funcalls': 64}
  " state: %s" % convergence_dict)


Done:  249


...

/home/karan/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 263.77518834, -327.57735321]), 'nit': 3, 'funcalls': 67}
  " state: %s" % convergence_dict)


Done:  253


/home/karan/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 274.93222192, -313.73875748]), 'nit': 0, 'funcalls': 21}
  " state: %s" % convergence_dict)
/home/karan/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 451.17236791, -432.36223478]), 'nit': 1, 'funcalls': 28}
  " state: %s" % convergence_dict)


Done:  254


/home/karan/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 531.87612348, -565.09174641]), 'nit': 4, 'funcalls': 70}
  " state: %s" % convergence_dict)
/home/karan/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-2796.09430504,   390.76735308]), 'nit': 4, 'funcalls': 56}
  " state: %s" % convergence_dict)
/home/karan/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 184.61249974,  304.76303997]), 'nit': 2, 'funcalls': 46}
  " state: %s" % convergence_dict)


In [69]:
from sklearn import svm
from sklearn.model_selection import train_test_split
clf = svm.SVC()
X_train, X_test, y_train, y_test = train_test_split(
    Xmod, ymod, test_size=0.25, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print "Incorrect classifications", np.sum(y_pred != y_test) , "out of", y_test.shape[0]

Incorrect classifications 2 out of 3


(array([-1,  1,  1]), array([ 1, -1,  1]))